In [1]:
import numpy as np
import codecs
import os
import glob
from numpy import log

In [2]:
# def loadfilee(rootpath, train_test, spam_ham):
#     result = glob.glob(rootpath + train_test + "/" + spam_ham + "/*.txt")
#     print('ls', len(result))
#     return result

In [3]:
#Load the datasets files and store them into a list
def loadfile(rootpath, train_test, spam_ham):
    result = os.listdir(rootpath + train_test + "/" + spam_ham + "/")
    for i in range(len(result)):
        result[i] = rootpath + train_test + "/" + spam_ham + "/" + result[i]
    return result

In [4]:
#Using the trainsets build the training models. Return a list contanins: 1).a dictionary with training words as keys and total number of words occurence as values.
#2). Total number of words in the training files.
def training(filelist):
    train_dict = {}
    # Step through all files in folder
    for file in filelist:
        # Read files, ignoring invalid
        words = open(file, 'r', encoding='utf-8', errors='ignore').read().rstrip().split()
        #Add number of occurences of each word as the values
        for word in words:
            if word in train_dict:
                train_dict[word] += 1
            else:
                train_dict[word] = 1
    total_num = 0
    for x in train_dict:
        total_num += train_dict[x]
        
    return [train_dict, total_num]

In [5]:
#Using the training model built before to test the testset data. Return a list of logP for files in each dataset. 
def testing(filelist, model, p, total_num):
    result = []
    for file in filelist:
        words = open(file, 'r', encoding='utf-8', errors='ignore').read().rstrip().split()
        re = log(p)
        for word in words:
            #Considering some words appear in testset but not in trainset, add a Laplace smoothing term.
            if word in model:
                re += log((model[word] + 1) / (total_num + len(model.keys())))
            else:
                re += log(1 / (total_num + len(model.keys())))
        result.append(re)
    return result

In [6]:
path = "/Users/longchen/Desktop/hw2/hw2_datasets/dataset 1/"
train_spam_list = loadfile(path, 'train', 'spam')
train_ham_list = loadfile(path, 'train', 'ham')

In [7]:
#Compute the prior probability of spam and ham trainsets
p_spam = len(train_spam_list) / (len(train_spam_list) + len(train_ham_list))
p_ham = len(train_ham_list) / (len(train_spam_list) + len(train_ham_list))

spam_weights = training(train_spam_list)
ham_weights = training(train_ham_list)

test_spam_list = loadfile(path, 'test', 'spam')
test_ham_list = loadfile(path, 'test', 'ham')

test_spam_spam = testing(test_spam_list, spam_weights[0], p_spam, spam_weights[1])
test_spam_ham = testing(test_spam_list, ham_weights[0], p_ham, ham_weights[1])
test_ham_spam = testing(test_ham_list, spam_weights[0], p_spam, spam_weights[1])
test_ham_ham = testing(test_ham_list, ham_weights[0], p_ham, ham_weights[1])

In [8]:
count = 0
#Count all correct classifications when predict == groundtruth
for i in range(len(test_spam_spam)):
    prediction = ''
    if test_spam_spam[i] > test_spam_ham[i]:
        prediction = 'spam'
    elif test_spam_spam[i] == test_spam_ham[i]:
        if p_spam > p_ham:
            prediction = 'spam'
        else:
            prediction = 'ham'
    else:
        prediction = 'ham'
        
    if prediction == 'spam':
        count += 1
        
for i in range(len(test_ham_spam)):
    prediction = ''
    if test_ham_spam[i] > test_ham_ham[i]:
        prediction = 'spam'
    elif test_ham_spam[i] == test_ham_ham[i]:
        if p_spam > p_ham:
            prediction = 'spam'
        else:
            prediction = 'ham'
    else:
        prediction = 'ham'
        
    if prediction == 'ham':
        count += 1
        
#compute the accuracy        
print(count / (len(test_spam_list) + len(test_ham_list)))

0.9372384937238494
